In [1]:
import yaml
import os
import zarr
import fibsem_tools as fst
import numpy as np

In [30]:
with open("../all_data.yaml") as f:
    datas = yaml.safe_load(f)

In [35]:
for dataset_name, dataset_info in datas["datasets"].items():
    print(dataset_name)
    if not os.path.exists(dataset_info["em"]["data"]):
        raise FileNotFoundError(f"Data file {dataset_info['data']} not found")
    if not os.path.exists(dataset_info["labels"]["data"]):
        raise FileNotFoundError(f"Labels file {dataset_info['labels']} not found")
    em_data = zarr.open(dataset_info["em"]["data"], "r")
    try:
        em_data[dataset_info["em"]["group"]]
    except KeyError:
        print(f"Key '{dataset_info['em']['group']}' not found in {em_data.path}")
        print(f"Available keys: {list(em_data.keys())}")
        raise
    labels_data = zarr.open(dataset_info["labels"]["data"], "r")
    try:
        crop_group = labels_data[dataset_info["labels"]["group"]]
        for crop in dataset_info["labels"]["crops"]:
            ll = crop_group[crop].attrs["cellmap"]["annotation"]["class_names"][0]
            print(crop_group[crop][ll]["s0"].shape)
    except KeyError:
        print(f"Key '{dataset_info['labels']['group']}' not found in {labels_data.path}")
        print(f"Available keys: {list(labels_data.keys())}")
        raise

jrc_cos7-1a
(400, 400, 400)
(400, 400, 400)
(385, 400, 400)
(299, 400, 400)
(400, 400, 400)
(1796, 1500, 2400)
(400, 400, 400)
(400, 400, 400)
(400, 400, 400)
(400, 400, 400)
(450, 600, 1100)
(400, 400, 400)
jrc_cos7-1b
(400, 400, 400)
(279, 900, 1600)
(400, 400, 400)
(400, 400, 400)
(285, 400, 400)
(243, 400, 400)
(400, 400, 400)
(400, 400, 400)
(400, 400, 400)
(400, 400, 400)
(400, 400, 400)
jrc_ctl-id8-1
(400, 400, 400)
(400, 400, 400)
(400, 400, 400)
(400, 400, 400)
(400, 400, 400)
jrc_fly-mb-1a
(400, 400, 400)
(400, 400, 400)
(400, 400, 400)
(400, 400, 400)
(400, 400, 400)
(400, 400, 400)
jrc_fly-vnc-1
(600, 600, 600)
(500, 500, 500)
(400, 600, 600)
(400, 400, 400)
(400, 400, 400)
(400, 600, 600)
jrc_hela-2
(200, 1000, 1000)
(500, 800, 800)
(476, 600, 600)
(500, 500, 500)
(160, 600, 600)
(200, 400, 400)
(106, 200, 200)
(220, 320, 320)
(130, 300, 300)
(128, 300, 300)
(400, 400, 400)
(220, 400, 400)
(110, 300, 300)
(500, 500, 500)
(400, 400, 400)
(400, 400, 400)
(400, 400, 400)
(400

In [ ]:
for key, crops in datas.items():
    print(key, "-", os.path.exists(f"/nrs/cellmap/data/{key}"), "-", os.path.exists(f"/nrs/cellmap/data/{key}/staging/groundtruth.zarr"))
    classes = set()
    for crop in crops:
        if not os.path.exists(f"/nrs/cellmap/data/{key}/staging/groundtruth.zarr/{crop}"):
            print(crop)
        else:
            crop_zarr = zarr.open(f"/nrs/cellmap/data/{key}/staging/groundtruth.zarr/{crop}", "r")
            classes = classes.union(set(crop_zarr.attrs["cellmap"]["annotation"]["class_names"]))
    print(classes)
            

In [19]:
with open("../data.yaml") as f:
    datas = yaml.safe_load(f)

In [6]:
root_gt_path = "/nrs/saalfeld/heinrichl/data/cellmap_labels/fly_organelles/"
root_data_path = "/nrs/cellmap/data/"

In [ ]:
for key, ds_info in datas.items():
    print(key, "-", os.path.exists(f"{root_gt_path}{key}"), "-", os.path.exists(f"{root_gt_path}{key}/groundtruth.zarr"), "-", os.path.exists(f"{ds_info['raw']}/s0"))
    for crop in ds_info["crops"]:
        if not os.path.exists(f"{root_gt_path}{key}/groundtruth.zarr/{crop}"):
            print(crop)

In [5]:
def fix_offset(data, crop, orig_offset, new_offset):
    diff_offset = np.array(orig_offset) - np.array(new_offset)
    crop_root = fst.access(f"{root_gt_path}{data}/groundtruth.zarr/{crop}", mode="r+")
    for lbl in list(crop_root.keys()):
        crop_label_root = crop_root[lbl]
        ms_attrs = crop_label_root.attrs["multiscales"]
        for ds in range(len(ms_attrs[0]["datasets"])):
            for k in range(len(ms_attrs[0]["datasets"][ds]["coordinateTransformations"])):
                if ms_attrs[0]["datasets"][ds]["coordinateTransformations"][k]["type"] == "translation":
                    transl = ms_attrs[0]["datasets"][ds]["coordinateTransformations"][k]["translation"] 
                    ms_attrs[0]["datasets"][ds]["coordinateTransformations"][k]["translation"] = list(np.array(transl) - diff_offset)
        crop_label_root.attrs["multiscales"] = ms_attrs

In [ ]:
# def fix_scale(data, crop):
#     crop_root = fst.access(f"{root_gt_path}{data}/groundtruth.zarr/{crop}", mode="r+")
#     for lbl in list(crop_root.keys()):
#         crop_label_root = crop_root[lbl]
#         ms_attrs = crop_label_rrot.attrs["multiscales"]
#         for ds in range(len(ms_attrs[0]["datasets"])):
#             for k in range(len(ms_attrs[0]["datasets"][ds]["coordinateTransformations"])):
#                 if ms_attrs[0]["datasets"][ds]["coordinateTransformations"][k]["type"] == "scale":
#                     scale = ms_attrs[0]["datasets"][ds]["coordinateTransformations"][k]["scale"]
#                     scale = [scale[1],]*3
#                     print(scale)
#                     #ms_attrs[0]["datasets"][ds]["coordinateTransformations"][k]["scale"]

In [ ]:
# for crop in datas["culture/jrc_ctl-id8-1"]["crops"]:
#     fix_scale("cutlure/jrc_ctl-id8-1", crop)

In [ ]:
for key, ds_info in datas.items():
    print(key)
    root_raw = fst.read(ds_info["raw"])
    try:
        datasets = root_raw.attrs["multiscales"][0]["datasets"]
    except KeyError as e:
        print("Different metadata format?")
        continue
    for ds in datasets:
        if ds["path"] == "s0":
            raw_res = ds["coordinateTransformations"][0]["scale"]
    raw_res_arr = np.array(raw_res)
    for crop in ds_info["crops"]:
        crop_root = fst.read(f"{root_gt_path}{key}/groundtruth.zarr/{crop}")
        lbl = list(crop_root.keys())[0]
        crop_datasets = crop_root[lbl].attrs["multiscales"][0]["datasets"]
        for cds in crop_datasets:
            if cds["path"] == "s0":
                crop_res = cds["coordinateTransformations"][0]["scale"]
                crop_off = cds["coordinateTransformations"][1]["translation"]
                crop_off_arr = np.array(crop_off)
                crop_res_arr = np.array(crop_res)
                corner_off_arr = (crop_off_arr + raw_res_arr/2. - crop_res_arr/2.)
                print("########################")
                if np.all(corner_off_arr % raw_res_arr == 0) and np.all(corner_off_arr % crop_res_arr == 0):
                    print(crop, "GOOD", raw_res, crop_res, crop_off)
                else:
                    print(crop, "BAD")
                    print("RAW -", raw_res, "; CROP -", crop_res, "@", crop_off,"->", corner_off_arr)
                    if np.all(crop_res < raw_res):
                        crop_off_arr_suggested = crop_off_arr - crop_res_arr/2.
                        corner_off_arr_suggested = (crop_off_arr_suggested + raw_res_arr/2. - crop_res_arr/2.)
                        if np.all(corner_off_arr_suggested % raw_res_arr == 0) and np.all(corner_off_arr_suggested % crop_res_arr == 0):
                            print("SUGGESTED WORKS:", "suggested offset:", list(crop_off_arr_suggested))
                            fix_offset(key, crop, crop_off, list(crop_off_arr_suggested))
                        else:
                            print("SUGGESTED STILL BAD")
                    else:
                        crop_off_arr_suggested = crop_off_arr + crop_res_arr/2. - raw_res_arr/2.
                        corner_off_arr_suggested = (crop_off_arr_suggested + raw_res_arr/2. - crop_res_arr/2.)
                        if np.all(corner_off_arr_suggested % raw_res_arr == 0) and np.all(corner_off_arr_suggested % crop_res_arr == 0):
                            print("SUGGESTED WORKS:", "suggested offset:", list(crop_off_arr_suggested))
                            fix_offset(key, crop, crop_off, list(crop_off_arr_suggested))
                        else:
                            print("SUGGESTED STILL BAD")
                        #print("IS DIVISIBLE: ", np.all(crop_off_arr%crop_res_arr == 0))
    print("---------------------------------------------------------------------------------------------------------------------------------------------")

In [ ]:
for key, ds_info in datas.items():
    print(key)
    root_raw = fst.read(ds_info["raw"])
    try:
        datasets = root_raw.attrs["multiscales"][0]["datasets"]
    except KeyError as e:
        print("Different metadata format?")
        continue
    for ds in datasets:
        if ds["path"] == "s0":
            raw_res = ds["coordinateTransformations"][0]["scale"]
    raw_res_arr = np.array(raw_res)
        
    for crop in ds_info["crops"]:
        crop_root = fst.read(f"{root_gt_path}{key}/groundtruth.zarr/{crop}")
        lbl = list(crop_root.keys())[0]
        crop_datasets = crop_root[lbl].attrs["multiscales"][0]["datasets"]
        for cds in crop_datasets:
            if cds["path"] == "s0":
                crop_res = cds["coordinateTransformations"][0]["scale"]
                
                crop_off = cds["coordinateTransformations"][1]["translation"]
                
                crop_off_arr = np.array(crop_off)
                crop_res_arr = np.array(crop_res)
                corner_off_arr = (crop_off_arr + raw_res_arr/2. - crop_res_arr/2.)
                if np.all(crop_res_arr > np.array([8.,8.,8.])):
                    print(f"crop {crop} has {crop_res}")
    print("---------------------------------------------------------------------------------------------------------------------------------------------")
                


In [ ]:
for key, ds_info in datas.items():
    print(key)
    root_raw = fst.read(ds_info["raw"])
    try:
        datasets = root_raw.attrs["multiscales"][0]["datasets"]
    except KeyError as e:
        print("Different metadata format?")
        continue
    for ds in datasets:
        if ds["path"] == "s0":
            raw_res = ds["coordinateTransformations"][0]["scale"]
    raw_res_arr = np.array(raw_res)
    offsets = []
    for crop in ds_info["crops"]:
        crop_root = fst.read(f"{root_gt_path}{key}/groundtruth.zarr/{crop}")
        lbl = list(crop_root.keys())[0]
        crop_datasets = crop_root[lbl].attrs["multiscales"][0]["datasets"]
        for cds in crop_datasets:
            if cds["path"] == "s0":
                crop_res = cds["coordinateTransformations"][0]["scale"]
                
                crop_off = cds["coordinateTransformations"][1]["translation"]
                
                print(f"{crop}:{crop_off}")
                offsets.append(tuple(crop_off))
    if len(set(offsets)) < len(offsets):
        print(f"This has some {len(set(offsets))}, {len(offsets)}")
    print("---------------------------------------------------------------------------------------------------------------------------------------------")
                


In [ ]:
for key, ds_info in datas.items():
    print(key)
    root_raw = fst.read(ds_info["raw"])
    try:
        datasets = root_raw.attrs["multiscales"][0]["coordinateTransformations"]
        if len(datasets) > 0:
            print(f"Check this one: {ds_info['raw']}")
    except KeyError as e:
        print("Different metadata format?")
        continue

In [ ]:
for key, ds_info in datas.items():
    print(key)
    root_raw = fst.read(ds_info["raw"])
    try:
        datasets = root_raw.attrs["multiscales"][0]["datasets"]
    except KeyError as e:
        print("Different metadata format?")
        continue
    for ds in datasets:
        if ds["path"] == "s0":
            raw_res = ds["coordinateTransformations"][0]["scale"]
    raw_res_arr = np.array(raw_res)
        
    for crop in ds_info["crops"]:
        crop_root = fst.read(f"{root_gt_path}{key}/groundtruth.zarr/{crop}")
        if "all" in crop_root:
            labels, counts = np.unique(crop_root["all"]["s0"], return_counts=True)
            if 0 in labels:
                print(f"{crop} labels: ", labels, "INCLUDES 0", counts)
            else:
                print(f" {crop} labels: ", labels)

In [ ]:
[ 1  2  8  9 10 11 16 17 35]